In [1]:
#Python-related imports
from datetime import datetime
import os

#PyData imports
import numpy as np
import pandas as pd
from scipy.optimize import bisect
import matplotlib
import matplotlib.pyplot as plt

#Torch-related imports
import torch
import torch.distributions as D
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Function
from tqdm.notebook import tqdm

In [2]:
# Hack to import from a parent directory
import sys
path = '..'
if path not in sys.path:
    sys.path.append(path)
    
from LogitNormal import *
from TruncatedNormal import *
from obs_and_flow import LowerBound
from SBM_SDE_classes_optim import *

In [3]:
torch.manual_seed(0)
np.random.seed(0)

torch.set_printoptions(precision = 8)

## Sample $\theta$
**Slower decay** parameters.

In [ ]:
temp_ref = 283
temp_rise = 5 #High estimate of 5 celsius temperature rise by 2100.

prior_scale_factor = 0.25

#Parameter prior means
u_M_mean = 0.0001
a_SD_mean = 0.5
a_DS_mean = 0.5
a_M_mean = 0.5
a_MSC_mean = 0.5
k_S_ref_mean = 0.00001625
k_D_ref_mean = 0.00005
k_M_ref_mean = 0.00003
Ea_S_mean = 20
Ea_D_mean = 20
Ea_M_mean = 20
c_SOC_mean = 0.01
c_DOC_mean = 0.001
c_MBC_mean = 0.002

#SCON theta logit-normal distribution parameter details in order of mean, sdev, lower, and upper.
u_M_details = torch.Tensor([u_M_mean, u_M_mean * prior_scale_factor, 0, 0.1])
a_SD_details = torch.Tensor([a_SD_mean, a_SD_mean * prior_scale_factor, 0, 1])
a_DS_details = torch.Tensor([a_DS_mean, a_DS_mean * prior_scale_factor, 0, 1])
a_M_details = torch.Tensor([a_M_mean, a_M_mean * prior_scale_factor, 0, 1])
a_MSC_details = torch.Tensor([a_MSC_mean, a_MSC_mean * prior_scale_factor, 0, 1])
k_S_ref_details = torch.Tensor([k_S_ref_mean, k_S_ref_mean * prior_scale_factor, 0, 0.1])
k_D_ref_details = torch.Tensor([k_D_ref_mean, k_D_ref_mean * prior_scale_factor, 0, 0.1])
k_M_ref_details = torch.Tensor([k_M_ref_mean, k_M_ref_mean * prior_scale_factor, 0, 0.1])
Ea_S_details = torch.Tensor([Ea_S_mean, Ea_S_mean * prior_scale_factor, 5, 80])
Ea_D_details = torch.Tensor([Ea_D_mean, Ea_D_mean * prior_scale_factor, 5, 80])
Ea_M_details = torch.Tensor([Ea_M_mean, Ea_M_mean * prior_scale_factor, 5, 80])

#SCON-C diffusion matrix parameter distribution details
c_SOC_details = torch.Tensor([c_SOC_mean, c_SOC_mean * prior_scale_factor, 0, 0.1])
c_DOC_details = torch.Tensor([c_DOC_mean, c_DOC_mean * prior_scale_factor, 0, 0.1])
c_MBC_details = torch.Tensor([c_MBC_mean, c_MBC_mean * prior_scale_factor, 0, 0.1])

In [ ]:
priors = {'u_M': u_M_details, 'a_SD': a_SD_details, 'a_DS': a_DS_details, 'a_M': a_M_details, 'a_MSC': a_MSC_details, 'k_S_ref': k_S_ref_details, 'k_D_ref': k_D_ref_details, 'k_M_ref': k_M_ref_details, 'Ea_S': Ea_S_details, 'Ea_D': Ea_D_details, 'Ea_M': Ea_M_details,
          'c_SOC': c_SOC_details, 'c_DOC': c_DOC_details, 'c_MBC': c_MBC_details}

Find logit normal (parent) loc and scale parameters that would roughly correspond to the target means and stddevs.

In [ ]:
def find_scale(scale, loc, a, b, target_sd):
    x = RescaledLogitNormal(loc, scale, a, b)
    #print(scale, x.mean, x.stddev)
    return x.stddev - target_sd

def sample_theta(priors):
    torch.manual_seed(0)
    scale_lower = 1e-8 #Lower bound for scale search by bisect function.
    scale_upper = 100 #Upper bound for scale search by bisect function. 
    
    theta_hyperparams = {} # hyperparams
    theta_samples = {} # theta samples
    for k, v in priors.items():
        sigmoid_loc, target_sd, a, b = v
        loc = logit(sigmoid_loc, a, b)
        scale = bisect(find_scale, scale_lower, scale_upper, (loc, a, b, target_sd))
        dist = RescaledLogitNormal(loc, scale, a, b)
        assert torch.abs(dist.stddev - target_sd) < 1e-5  
        
        theta_hyperparams[k] = torch.tensor((loc, scale, a, b))
        theta_samples[k] = dist.sample()
        
    return theta_hyperparams, theta_samples

In [ ]:
theta_hyperparams, theta_samples = sample_theta(priors)

Plot sampled theta and its prior distribution.

In [ ]:
keys = list(theta_hyperparams.keys())
loc, scale, a, b = torch.tensor(list(zip(*(theta_hyperparams[k] for k in keys))))
loc, scale, a, b

In [ ]:
plot_means, plot_sds, _, _ = torch.tensor(list(zip(*(priors[k] for k in keys))))
dist = RescaledLogitNormal(loc, scale, a, b)
x0 = torch.max(plot_means - 4*plot_sds, a).detach()
x1 = torch.min(plot_means + 4*plot_sds, b).detach()

num_pts = 1000
x = torch.from_numpy(np.linspace(x0, x1, num_pts))
pdf = torch.exp(dist.log_prob(x)).detach()

In [ ]:
# Plot sampled theta values
fig, axes = plt.subplots(4, 4, figsize=(15, 15))
k = 0
for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        if k < 14:
            key = keys[k]
            ax.plot(x[:, k], pdf[:, k])
            ax.axvline(theta_samples[key], color='gray')
            ax.set_xlabel(key)
            ax.set_ylabel('density')
        else:
            fig.delaxes(axes[i, j])
        k += 1  
plt.tight_layout()

In [5]:
temp_ref = 283
temp_rise = 5 #High estimate of 5 celsius temperature rise by 2100.

load_dir = 'data/dt_01_t_1000000_minibatch_logit_theta_trunc_trans/SCON_C_slow'
hyperparams_file = '{}_hyperparams.pt'.format(load_dir)
theta_file = '{}_theta.pt'.format(load_dir)
x_file = '{}_x.pt'.format(load_dir)
x0_dist_file = '{}_x0_dist.pt'.format(load_dir)

theta_hyperparams = torch.load(hyperparams_file)
theta_samples = torch.load(theta_file)
x = torch.load(x_file)
p_x0 = torch.load(x0_dist_file)

Alternatively, load **faster decay** parameters from previous experiments.

In [ ]:
temp_ref = 283
temp_rise = 5 #High estimate of 5 celsius temperature rise by 2100.

# Use the same theta priors and samples to ensure fair comparison
hyperparams_file = 'data/dt_1_t_100000_n_1_minibatch/theta_from_multi_x_hyperparams_scon_c_{}.pt'.format(0)
theta_file = 'data/dt_1_t_100000_n_1_minibatch/theta_from_multi_x_theta_scon_c_{}.pt'.format(0)
theta_hyperparams = torch.load(hyperparams_file)
theta_samples = torch.load(theta_file)

Compare:
- Normal transition with naive zero-thresholding
- TruncatedNormal transition

## Sample $x, y|\theta$

In [37]:
def generate_data(SBM_SDE_CLASS, DIFFUSION_TYPE, X0_LOC, X0_SCALE, T, DT, THETA_DICT, I_S_FUNC, I_D_FUNC, TEMP_FUNC, TEMP_REF, TEMP_RISE, OBS_EVERY, OBS_ERROR_SCALE, seed=0):    
    torch.manual_seed(seed)
    
    # Vectorize variable calculations where possible
    N = int(T / DT) + 1
    hours = torch.tensor(np.linspace(0, T, N), dtype=torch.float)
    I_S_tensor = I_S_FUNC(hours)
    I_D_tensor = I_D_FUNC(hours)
    temps = TEMP_FUNC(hours, TEMP_REF, TEMP_RISE)
    model = SBM_SDE_CLASS(hours, I_S_tensor, I_D_tensor, temps, TEMP_REF, DIFFUSION_TYPE)
    
    # Draw initial condition x0 ~ N(loc=X0_LOC, scale=X0_LOC * X0_SCALE)
    x = torch.empty([N, model.state_dim])
    X0_LOC = torch.as_tensor(X0_LOC)
    p_x0 = TruncatedNormal(loc = X0_LOC, scale = X0_LOC * X0_SCALE, a = 0, b = float('inf'))
    x0_samples = p_x0.sample() # (state_dim, )
    print('X0_samples = ', x0_samples)
    x[0, :] = x0_samples
    
    #Take Euler-Maruyama step. 
    for i in tqdm(range(1, N)):
        # Define x_i ~ N(loc = x_prev + alpha(x_prev, theta)*dt, scale = sqrt(beta(x_prev, theta)*dt))
        a = model.calc_drift(x[i - 1, :], THETA_DICT, I_S_tensor[i], I_D_tensor[i], temps[i])
        b = model.calc_diffusion_sqrt(x[i - 1, :], THETA_DICT, diffusion_matrix=False)
        #a = ALPHA(x[:, i - 1, :], THETA_DICT, I_S_tensor[i], I_D_tensor[i], temps[i], TEMP_REF, arrhenius_temp_dep, linear_temp_dep)
        #b = BETA(x[:, i - 1, :], THETA_DICT, diffusion_matrix=False)
        p_x_i = TruncatedNormal(loc = x[i - 1, :] + a * DT, scale = b * math.sqrt(DT), a = 0, b = float('inf'))
        
        # Draw sample
        x[i, :] = p_x_i.sample()
        
    #return {'x': x, 't_x': np.arange(0, T + DT, DT), 'p_x0': p_x0, 'model': model}
        
    # Compute CO2
    x_for_y = x[0::int(OBS_EVERY / DT), :] #Slice x based on observation interval to generate y.
    CO2 = model.calc_CO2(x_for_y, THETA_DICT, temps[::int(OBS_EVERY / DT)].unsqueeze(-1)) # after or before subsampling, can self.temps, etc. work for subsampled time steps?
    x_with_CO2_for_y = torch.cat((x_for_y, CO2), dim=-1) # (obs_len, state_dim + 1)
    print('x_with_CO2_for_y:', x_with_CO2_for_y.shape)
    
    # Generate y ~ N(x, OBS_ERROR_SCALE * mean(x))
    obs_scale = OBS_ERROR_SCALE * x_with_CO2_for_y.mean(0) # (state_dim + 1, )
    y = TruncatedNormal(x_with_CO2_for_y, obs_scale.reshape((1, -1)), 0, float('inf')).sample()
    return {'y': y, 't_y': np.arange(0, T + DT, OBS_EVERY), 'y_std': obs_scale, 'x': x, 't_x': np.arange(0, T + DT, DT),
            'CO2': CO2, 't_CO2': np.arange(0, T + DT, OBS_EVERY), 'p_x0': p_x0}

In [42]:
dt = 0.1
t = 1000000
x0_SCON = [65, 0.4, 2.5]
obs_every = 5
obs_error_scale = 0.1

In [43]:
y_dict = generate_data(SCON_optim, 'C', x0_SCON, obs_error_scale, t, dt, theta_samples, i_s, i_d, temp_gen, temp_ref, temp_rise, obs_every, obs_error_scale)

X0_samples =  tensor([64.93901062,  0.42932013,  2.16245246])



torch.Size([200001, 1])
torch.Size([200001, 1])
x_with_CO2_for_y: torch.Size([200001, 4])


In [45]:
y_dict['y'].shape, y_dict['x'].shape

(torch.Size([200001, 4]), torch.Size([10000001, 3]))

In [9]:
x_for_y = x[0::int(obs_every / dt), :]
x_for_y.shape

torch.Size([200001, 3])

In [27]:
CO2 = model.calc_CO2(x_for_y, theta_samples, model.temps[::int(obs_every / dt)].unsqueeze(-1))

torch.Size([200001, 1])
torch.Size([200001, 1])


In [28]:
CO2.shape

torch.Size([200001, 1])

In [14]:
model.temps

tensor([283.00000000, 283.26251221, 283.52478027, 283.78674316, 284.04815674,
        284.30886841, 284.56863403, 284.82739258, 285.08486938, 285.34091187,
        285.59536743])

In [ ]:
def generate_y(x, model, T, DT, THETA_DICT, OBS_EVERY, OBS_ERROR_SCALE, seed=0):
    torch.manual_seed(seed)
    
    # Compute CO2
    x_for_y = x[0::int(OBS_EVERY / DT), :] #Slice x based on observation interval to generate y.
    CO2 = model.calc_CO2(x_for_y, THETA_DICT, model.temps[::int(OBS_EVERY / DT)].unsqueeze(-1)) # after or before subsampling, can self.temps, etc. work for subsampled time steps?
    x_with_CO2_for_y = torch.cat((x_for_y, CO2), dim=-1) # (n, state_dim + 1)
    
    # Generate y ~ N(x, OBS_ERROR_SCALE * mean(x))
    obs_scale = OBS_ERROR_SCALE * x_with_CO2_for_y.mean(0) # (state_dim, )
    y = TruncatedNormal(x_with_CO2_for_y, obs_scale.reshape((1, -1)), 0, float('inf')).sample()
    return {'y': y, 't_y': np.arange(0, T + DT, OBS_EVERY), 'y_std': obs_scale, 'x': x, 't_x': np.arange(0, T + DT, DT),
            'CO2': CO2, 't_CO2': np.arange(0, T + DT, OBS_EVERY), 'p_x0': p_x0}

In [ ]:
def plot_path(x, y, t_x, t_y, title=None):
    time_steps, state_dim = x.shape
    fig, axes = plt.subplots(state_dim + 1, figsize=(15, 15), sharex=True)
    
    labels = ['SOC', 'DOC', 'MBC', 'CO2']
    colors = ['m', 'c', 'g', 'orange']
    for i, ax in enumerate(axes):            
        # Only plot observations for CO2 (since it is not a state)
        if i < state_dim:
            ax.plot(t_x, x[:, i], color=colors[i], label='latent state')
        ax.plot(t_y, y[:, i], alpha=0.3, color=colors[i], label='obervation')
        
        ax.set_ylabel(labels[i])
        ax.legend()
        if i == 0 and title:
            ax.set_title(title)
    ax.set_xlabel('hour')
    plt.tight_layout()

In [ ]:
x, t_x = y_dict['x'], y_dict['t_x']
y, t_y = y_dict['y'], y_dict['t_y']
plot_path(x, y, t_x, t_y, 'SCON-C')

In [ ]:
def plot_theta(theta_dict, hyperparams_dict):
    keys = list(hyperparams_dict.keys())
    loc, scale, a, b = torch.tensor(list(zip(*(hyperparams_dict[k] for k in keys))))

    dist = RescaledLogitNormal(loc, scale, a, b)
    x0 = torch.max(dist.mean - 4*dist.stddev, a).detach()
    x1 = torch.min(dist.mean + 4*dist.stddev, b).detach()
    
    num_pts = 1000
    x = torch.from_numpy(np.linspace(x0, x1, num_pts))
    pdf = torch.exp(dist.log_prob(x)).detach()
    
    # Save sampled theta values
    fig, axes = plt.subplots(4, 4, figsize=(15, 15))
    k = 0
    for i, row in enumerate(axes):
        for j, ax in enumerate(row):
            if k < 14:
                key = keys[k]
                ax.plot(x[:, k], pdf[:, k])
                ax.axvline(theta_dict[key], color='gray')
                ax.set_xlabel(key)
                ax.set_ylabel('density')
            else:
                fig.delaxes(axes[i, j])
            k += 1  
    plt.tight_layout()

In [ ]:
plot_theta(theta_samples, theta_hyperparams)

In [ ]:
save_dir = 'data/dt_01_t_1000000_minibatch_logit_theta_trunc_trans'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
save_str = '{}/SCON_C_slow'.format(save_dir)
hyperparams_file = '{}_hyperparams.pt'.format(save_str)
theta_file = '{}_theta.pt'.format(save_str)
x_file = '{}_x.pt'.format(save_str)
x0_dist_file = '{}_x0_dist.pt'.format(save_str)
#y_dict_file = '{}_y_dict.pt'.format(save_str)

torch.save(theta_hyperparams, hyperparams_file)
torch.save(theta_samples, theta_file)
torch.save(x_dict['x'], x_file)
torch.save(x_dict['p_x0'], x0_dist_file)
#torch.save(y_dict, y_dict_file)

#theta_hyperparams = torch.load(hyperparams_file)
#theta_samples = torch.load(theta_file)
#x = torch.load(x_file)
#p_x0 = torch.load(x0_dist_file)

In [ ]:
x.shape

In [ ]:
#Save CSV of stochastic path.
df_y = pd.DataFrame(data = {'hour': y_dict['t_y'], 'SOC': y_dict['y'][0, :], 'DOC': y_dict['y'][1, :], 'MBC': y_dict['y'][2, :], 'CO2': y_dict['y'][3, :]})
df_y.to_csv(save_string + '.csv', index = False)

In [ ]:
print(np.max(df_y['SOC']))
print(np.max(df_y['DOC']))
print(np.max(df_y['MBC']))
print(np.max(df_y['CO2']))

In [ ]:
#Save rsampled theta values.
torch.save(SCON_SS_params_dict, save_string + '_rsample.pt')

#Save priors dict.
torch.save(SCON_SS_priors_dict, save_string + '_hyperparams.pt')

In [ ]:
print(df_y)

## Archive

In [ ]:
y_det_dict = get_SBM_SDE_euler_maruyama_y_det(alpha_SCON, x0_SCON, t, dt, SCON_SS_params_dict, I_S_func, I_D_func, temp_func, temp_ref, temp_rise, obs_every, obs_error_scale)

In [ ]:
fig2, axs2 = plt.subplots(4, sharex = True)
axs2[0].plot(y_det_dict['t_x'], y_det_dict['x'][0, :], color = "m", label = 'SOC x')
axs2[0].scatter(y_det_dict['t_y'], y_det_dict['y'][0, :], color = "m", alpha = 0.3, label = 'SOC y')
axs2[1].plot(y_det_dict['t_x'], y_det_dict['x'][1, :], color = "c", label = 'DOC x')
axs2[1].scatter(y_det_dict['t_y'], y_det_dict['y'][1, :], color = "c", alpha = 0.3, label = 'DOC y')
axs2[2].plot(y_det_dict['t_x'], y_det_dict['x'][2, :], color = "g", label = 'MBC x')
axs2[2].scatter(y_det_dict['t_y'], y_det_dict['y'][2, :], color = "g", alpha = 0.3, label = 'MBC y')
axs2[3].plot(y_det_dict['t_x'], y_det_dict['x'][3, :], color = "orange", label = 'CO2')
axs2[3].scatter(y_det_dict['t_y'], y_det_dict['y'][3, :], color = "orange", alpha = 0.3, label = 'CO2 y')

In [ ]:
save_string_det = dir_path + f'{sbm_model}_sample_det_y_t_{t}_dt_{dt}_sd_scale_{prior_scale_factor}'.replace('.','-')
fig2.savefig(save_string_det + '.png', dpi = 300)

In [ ]:
df_y_det = pd.DataFrame(data = {'hour': y_det_dict['t_y'], 'SOC': y_det_dict['y'][0, :], 'DOC': y_det_dict['y'][1, :], 'MBC': y_det_dict['y'][2, :], 'CO2': y_det_dict['y'][3, :]})
df_y_det.to_csv(save_string_det + '.csv', index = False)

In [ ]:
print(df_y_det)

In [ ]:
TruncatedNormal(0, 1, 0, float('inf'))